In [9]:
''' 
    Impor Pustaka apa yang kita butuhkan
'''
import cv2
import mediapipe as mp
import numpy as np
from datetime import datetime
import time

In [10]:
''' 
    Config Mediapipe Solution
'''
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence = 0.5, min_tracking_confidence=0.5)

mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

In [11]:
''' 
    Pojok Dir => Tempat untuk memeriksa metode/atribut
'''
print(dir(mp_face_mesh))
# print(dir(cv2))

['FACEMESH_CONTOURS', 'FACEMESH_FACE_OVAL', 'FACEMESH_IRISES', 'FACEMESH_LEFT_EYE', 'FACEMESH_LEFT_EYEBROW', 'FACEMESH_LEFT_IRIS', 'FACEMESH_LIPS', 'FACEMESH_NOSE', 'FACEMESH_NUM_LANDMARKS', 'FACEMESH_NUM_LANDMARKS_WITH_IRISES', 'FACEMESH_RIGHT_EYE', 'FACEMESH_RIGHT_EYEBROW', 'FACEMESH_RIGHT_IRIS', 'FACEMESH_TESSELATION', 'FaceMesh', 'NamedTuple', 'SolutionBase', '_BINARYPB_FILE_PATH', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'association_calculator_pb2', 'constant_side_packet_calculator_pb2', 'detections_to_rects_calculator_pb2', 'gate_calculator_pb2', 'image_to_tensor_calculator_pb2', 'inference_calculator_pb2', 'landmarks_refinement_calculator_pb2', 'logic_calculator_pb2', 'non_max_suppression_calculator_pb2', 'np', 'rect_transformation_calculator_pb2', 'split_vector_calculator_pb2', 'ssd_anchors_calculator_pb2', 'tensors_to_classification_calculator_pb2', 'tensors_to_detections_calculator_pb2', 'tensors_to_landmarks_c

In [12]:
nose_tip = [1]
right_eye = [33]
left_eye = [263]
lips_upper = [61, 291]
chin = [199]
targetted_landmark = []

targetted_landmark.extend(item for sublist in [nose_tip, right_eye, left_eye, lips_upper, chin] for item in sublist)

looking_down_timer = None
required_looking_time = 2

In [13]:
'''
    Define Landmark yang kita butuhkan
'''
chosen_left_eye_idxs  = [362, 385, 387, 263, 373, 380]
chosen_right_eye_idxs = [33,  160, 158, 133, 153, 144]
chosen_mouth_idxs = [
    #upper right mouth
    82, 38, 72, 37,
    81, 41, 73, 39,
    80, 42, 74, 40,
    183,184, 185, 191,
    61, 76, 62, 78,

    #upper middle mouth
    14,  13, 0, 11,

    #upper left mouth
    312, 268, 302, 267,
    311, 271, 303, 269,
    310, 272, 304, 270,
    415, 407, 408, 409,
    291, 292, 308, 306,
    324,325, 307, 375,

    #botom right mouth
    146, 77,
    88, 89,90, 91,
    178, 179, 180, 181,
    87, 86, 85, 84,

    #botom middle mouth
    12, 15, 16, 17,

    #botom left mouth
    318, 319, 320, 321,
    402, 403, 404, 405,
    317, 316, 315, 314
    ]
nose_idxs = [1]
forehead_idxs = [8]
our_landmarks = []

for idxs in (chosen_left_eye_idxs, chosen_right_eye_idxs, chosen_mouth_idxs, nose_idxs, forehead_idxs):
    our_landmarks.extend(idxs)

In [14]:
def diffTime(looking_down_timer):
    diff = (datetime.now() - looking_down_timer).total_seconds()
    return diff 

def putText(frame, val, x, y, unity=""):
    if len(unity) == 0:
        cv2.putText(frame, f'{val}', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    else:
        cv2.putText(frame, f'{val:.4f} {unity}', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

In [15]:
''' 
    Baca Kamera
'''
cap = cv2.VideoCapture(0)

# Loop utama untuk memproses setiap frame dari kamera
while cap.isOpened():
    start = time.time()
    # Baca frame terbaru dari kamera
    success, image = cap.read()
    # Balikkan gambar secara horizontal dan konversi ke RGB
    image = cv2.cvtColor(cv2.flip(image,1), cv2.COLOR_BGR2RGB)
    # Optimalkan kinerja dengan mengubah flag writeable gambar
    image.flags.writeable = False
    # Proses deteksi wajah dan landmark dengan MediaPipe
    results = face_mesh.process(image)
    # Proses deteksi wajah dan landmark dengan MediaPipe
    image.flags.writeable = True
    # Konversi gambar kembali ke format BGR
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Mendapatkan dimensi gambar
    img_h, img_w, img_c = image.shape
    face_3d = []
    face_2d = []
    
    
    # Cek apakah terdapat wajah yang terdeteksi
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            for idx, lm in enumerate(face_landmarks.landmark):
                if idx in targetted_landmark:
                    if idx == 1:
                    # Dapatkan koordinat hidung dalam 2D dan 3D
                        nose_2d = (lm.x * img_w, lm.y * img_h)
                        nose_3d = (lm.x * img_w, lm.y * img_h, lm.z * 3000)
                    
                    x, y = int(lm.x * img_w), int(lm.y * img_h)
                    
                    # Dapatkan koordinat 2D
                    face_2d.append([x, y])
                    # Dapatkan koordinat 3D
                    face_3d.append([x, y, lm.z])
                    
            # Konversi koordinat menjadi array NumPy
            face_2d = np.array(face_2d, dtype=np.float64)
            face_3d = np.array(face_3d, dtype=np.float64)
            
            # Matriks kamera
            focal_length = 1 * img_w
            cam_matrix = np.array([
                                    [focal_length, 0, img_h / 2],
                                    [0, focal_length, img_w / 2],
                                    [0, 0, 1]
                                ])
            dist_matrix = np.zeros((4, 1), dtype=np.float64)
            
            # Hitung orientasi kepala menggunakan solvePnP
            success, rot_vec, trans_vec = cv2.solvePnP(face_3d, face_2d, cam_matrix, dist_matrix)
            # Matriks Rotasi
            rmat, jac = cv2.Rodrigues(rot_vec)
            # Angle
            angles, mtxR, mtxQ, Qx, Qy, Qz = cv2.RQDecomp3x3(rmat)
            
            # Mendapatkan Derajat Rotasi Y
            x = angles[0] * 360
            y = angles[1] * 360
            z = angles[2] * 360
            
            # Melihat kemiringan kepala pengguna
            if y < -10:
                text = 'Looking Left'
            elif y > 10:
                text = 'Looking Right'
            elif x < -7:
                text = 'Looking Down'
            elif x > 10:
                text = 'Looking Up'
            else:
                text = 'Forward'
            
            if text == 'Looking Down' and looking_down_timer == None:
                looking_down_timer = datetime.now()
            elif text != 'Looking Down': 
                looking_down_timer = None
                
            if looking_down_timer is not None :
                diff_time = diffTime(looking_down_timer)
                putText(image, diff_time, 20, 100, 'seconds')
            
                if diff_time > required_looking_time:
                    putText(image, "GO SLEEP NOW", 20, 150)
                print(looking_down_timer, '\t', diff_time)
                
            # Memproyeksikan koordinat hidung ke 3d
            nose_3d_projection, jacobian = cv2.projectPoints(nose_3d, rot_vec, trans_vec, cam_matrix, dist_matrix)
            # Gambar Arah Pandangan
            p1 = (int(nose_2d[0]), int(nose_2d[1]))
            p2 = (int(nose_2d[0] + y * 10), int(nose_2d[1] - x * 10))
            cv2.line(image, p1, p2, (255, 0, 0), 3)
            
            
            # Tambahkan text pada gambar
            putText(image, text, 20, 50)
            putText(image, x, 450, 50, "x")
            putText(image, y, 450, 100, "y")
            putText(image, z, 450, 150, "z")
            
            # Hitung FPS
            end = time.time()
            totalTime = end - start
            fps = 1 / totalTime
            putText(image, fps, 20, 450, "fps")
            
            
            # Draw landmarks on the frame
            filtered_landmarks = [face_landmarks.landmark[idx] for idx in our_landmarks]
            for landmark in filtered_landmarks:
                x, y = int(landmark.x * image.shape[1]), int(landmark.y * image.shape[0])
                cv2.circle(image, (x, y), 2, (0, 255, 0), -1)

            
            cv2.imshow('Head Pose Estimation', image)
            
            if cv2.waitKey(1) == ord('q'):
                break
            
cap.release()

2023-08-19 15:25:32.603566 	 0.0
2023-08-19 15:25:32.603566 	 0.030983
2023-08-19 15:25:32.603566 	 0.061696
2023-08-19 15:25:32.603566 	 0.107749
2023-08-19 15:25:32.603566 	 0.138661
2023-08-19 15:25:32.603566 	 0.170388
2023-08-19 15:25:32.603566 	 0.20062
2023-08-19 15:25:32.603566 	 0.231103
2023-08-19 15:25:32.603566 	 0.26408
2023-08-19 15:25:32.603566 	 0.310843
2023-08-19 15:25:32.603566 	 0.340923
2023-08-19 15:25:32.603566 	 0.372452
2023-08-19 15:25:32.603566 	 0.403967
2023-08-19 15:25:32.603566 	 0.433811
2023-08-19 15:25:32.603566 	 0.466782
2023-08-19 15:25:32.603566 	 0.497313
2023-08-19 15:25:32.603566 	 0.529005
2023-08-19 15:25:32.603566 	 0.560046
2023-08-19 15:25:32.603566 	 0.591327
2023-08-19 15:25:32.603566 	 0.622465
2023-08-19 15:25:32.603566 	 0.669306
2023-08-19 15:25:32.603566 	 0.715672
2023-08-19 15:25:32.603566 	 0.746034
2023-08-19 15:25:32.603566 	 0.775972
2023-08-19 15:25:32.603566 	 0.807506
2023-08-19 15:25:32.603566 	 0.840094
2023-08-19 15:25:32

KeyboardInterrupt: 

In [ ]:
cap.release()   
cv2.destroyAllWindows()